# Use `spacy-llm`  to add model predictions to Argilla Feedback

In this tutorial, we'll implement a `spacy-llm` pipeline to obtain model predictions with GPT3.5 and add them to our `FeedbackDataset` as suggestions.

Let us first install the required libraries for our task,

In [ ]:
pip install "spacy-llm[transformers]" "transformers[sentencepiece]" argilla datasets -qqq

and import them as well.

In [ ]:
import os
import spacy
from spacy_llm.util import assemble
import argilla as rg
import datasets

To be able to use GPT3.5 and other models from OpenAI, we'll need an API key from [openai.com](https://openai.com) and set it as an environment variable.

In [ ]:
os.environ["OPENAI_API_KEY"] = "<YOUR_OPEN_AI_API_KEY>"

In [ ]:
There are two ways to implement a `spacy-llm` pipeline for your LLM task: running the pipeline in the source code or using a `config.cfg` file to define all the parts of your pipeline. In this tutorial, we'll work with a config file and you can have more info about running directly in Python [here](https://spacy.io/usage/large-language-models#example-3).
We can create and save a default config file with `init config`

In [ ]:
spacy init config config.cfg

Now going into the `config.cfg`, you can change it to the settings below. With these settings, we create an LLM pipeline for text classification in English with GPT3.5.

`spacy-llm` offers various models to implement in your pipeline. You can have a look at the available [OpenAI models](https://spacy.io/api/large-language-models#models-rest) as well as checking the [HuggingFace models](https://spacy.io/api/large-language-models#models-hf) offered if you want to work with open-source models.

In [ ]:
[nlp]
lang = "en"
pipeline = ["llm"]

[components]

[components.llm]
factory = "llm"

[components.llm.task]
@llm_tasks = "spacy.TextCat.v2"
labels = ["POLITICS", "SPORTS"]

[components.llm.model]
@llm_models = "spacy.GPT-3-5.v1"

And, assemble the config file.

In [ ]:
nlp = assemble("config.cfg")

Now, we are ready to make inferences with the pipeline we have created.

In [ ]:
doc = nlp("No matter how hard they tried, Barcelona lost the match.")
print(doc.cats)

## FeedbackDataset

Now that we have our pipeline for inference, we can create our Argilla `FeedbackDataset` to make and store model predictions.